In [1]:
import os
import re
import csv
import time
import string
import requests
import numpy as np
import pandas as pd
import fitz  # PyMuPDF
from datetime import date, datetime
from requests.exceptions import TooManyRedirects, ConnectionError, ReadTimeout
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup

Funciones para realizar el web scrapping y almacenar la información en archivos de texto (descargar resultados de carreras):

In [ ]:
def status():
  """Función que analiza si existe el directorio backups, y de ser así importa los CSV correspondientes para el
  funcionamiento del programa. Si el directorio no existe, se asume que es la primera ejecución del programa y se declaran
  todas las variables necesarias sin datos. También se evalúa si durante la ejecución anterior se interrumpió el proceso
  de descarga de carreras  inesperadamente (fallas de conexión, corte de energía, etc),  y en caso afirmativo activa
  la corrección de errores.
  """

  try:
    if os.listdir('./backups'):
      directorios = True

      lista_existentes = []

      for c in os.listdir('./backups/carreras'): # Se crea una lista con todos los IDs descargados históricamente.
        c = c.replace('.txt','')
        lista_existentes.append(int(c))

      max_existente = max(lista_existentes) # Se almacenan estos datos como parte de la información a brindar al usuario durante la ejecución.
      min_existente = min(lista_existentes)

      with open('./backups/en_proceso.csv', mode='r', newline='') as archivo_csv:
        lector = csv.reader(archivo_csv)
        en_proceso = next(lector)
        en_proceso = bool(en_proceso[0])  # Según sea el contenido del archivo en_proceso.csv, se asigna valor booleano a la variable.

      with open('./backups/fallidos.csv', mode='r', newline='') as archivo_csv:
        lector = csv.reader(archivo_csv)
        lista_fallidos = next(lector) # Esta lista contiene todos los IDs de carreras que no fueron encontrados hasta la fecha.

      with open('./backups/ultima_act_fecha.csv', mode='r', newline='') as archivo_csv:
        lector = csv.reader(archivo_csv)
        fecha_list = next(lector) # Dato informativo para el usuario, sobre última fecha de actualización de la base de resultados.

      fecha_str = ''.join(fecha_list)
      fecha_obj = datetime.strptime(fecha_str, '%d-%m-%Y')
      fecha = fecha_obj.strftime('%d-%m-%Y') # Se formatea la fecha a formato datetime.

      with open('./backups/ultima_act_tiempo.csv', mode='r', newline='') as archivo_csv:
        lector = csv.reader(archivo_csv)
        tiempo = next(lector)  # Indica cuánto tiempo demoró la última actualización.

  # Si no encuentra el directorio especificado en el try, se asume que es la primera ejecución
  # y todas las variables se declaran sin asignarles valor útil. Se crea también el directorio 'calendario'.

  except OSError:
      directorios = False
      lista_fallidos = 'Sin datos'
      fecha = 'Sin datos'
      tiempo = 'Sin datos'
      min_existente = 'Sin datos'
      max_existente = 'Sin datos'
      lista_existentes = 'Sin datos'
      en_proceso = None
      os.mkdir('./calendario')

  return directorios, lista_fallidos, fecha, tiempo, min_existente, max_existente, lista_existentes, en_proceso

In [3]:
def comprobar_errores(lista_existentes, min_existente, max_existente):
    """Esta función es activada por la función integradora d_c, definida más adelante. Si la variable 'en_proceso' tiene
    valor True, significa que durante la última ejecución el proceso se interrumpió inesperadamente, durante la descarga
    de nuevas carreras o reintentos de fallidas, sin llegar a actualizar el CSV correspondiente, por lo tanto pudiéndose
    haber generado inconsistencias en la información. De ser el caso, la función procede a realizar una revisión de las
    listas pertinentes y rectificar los registros antes de continuar. Por último, en caso de que la interrupción se haya
    generado durante la primera ejecución del programa, verifica si existe el archivo procesados.csv y df_resultados.csv,
    y los crea en caso de que no existan.

    Argumentos:

    lista_existentes -- Lista de todos los ID descargados, generada por la función status().
    min_existente -- Mínimo ID descargado, obtenido por la función status().
    max_existente -- Máximo ID descargado , obtenido por la función status().

    """

    def corregir_lista_fallidos(lista_existentes, min_existente, max_existente):

        lista_fallidas_correccion = []

        for i in range(min_existente, max_existente):  # Se recorren los IDs existentes entre el mínimo y máximo, y se evalúa si existen en la lista de existentes.
          if i not in lista_existentes:
            lista_fallidas_correccion.append(i) # Si no se encuentran la lista de existentes, se asume que es fue un intento fallido, por lo cual se agrega a esta última lista.

        with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
          escritor = csv.writer(archivo_csv)
          escritor.writerow(lista_fallidas_correccion)  # Se actualiza el archivo de respaldo.

        print('Lista de fallidos verificada.')
        time.sleep(0.2)

        return lista_fallidas_correccion

    def verificar_procesados_y_resultados():
        if not os.path.exists('./backups/procesados.csv'): # Comportamiento en caso de no encontrarse el archivo procesados.csv

            procesados = []
            with open('./backups/procesados.csv', mode='w', newline='') as archivo_csv:
                escritor = csv.writer(archivo_csv)
                escritor.writerow(procesados)
            print('Lista de carreras procesadas generada (primera ejecución)')
            time.sleep(0.2)
        else:
          print('No se encontraron problemas con la lista de carreras procesadas.')
          time.sleep(0.2)

        if not os.path.exists('./backups/df_resultados.csv'): # Comportamiento en caso de no encontrarse el df_resultados.csv
            df_resultados = pd.DataFrame({
                'id':[],
                'nombre':[],
                'fecha':[],
                'hora':[],
                'metros':[],
                'suelo':[],
                'condicion':[],
                'posicion':[],
                'numero':[],
                'competidor':[],
                'diferencia':[],
                'peso_jockey':[],
                'peso_caballo':[],
                'pagaria':[]
                })
            df_resultados.to_csv('./backups/df_resultados.csv')
            print('Base de datos de resultados generada (primera ejecución)')
            time.sleep(0.2)
        else:
          print('No se encontraron problemas con la base de datos de resultados.')
          time.sleep(0.2)

    lista_fallidos_corregida = corregir_lista_fallidos(lista_existentes, min_existente, max_existente)
    verificar_procesados_y_resultados()

    return lista_fallidos_corregida

In [4]:
def descargar_carreras(directorios, lista_fallidos, fecha, tiempo, min_existente, max_existente, lista_existentes):
    """Según sea la primera ejecución o no (if directorios == True/False), genera los directorios necesarios y comienza
    la primera descarga de resultados de carreras

    Argumentos: Son generados por la función status() y/o eventualmente por la función comprobar_errores().
    """

    if directorios == True:
            print('- Actualización de archivos -')

            print(f"Actualmente existen {len(lista_fallidos)} ID's de carreras no encontrados")
            time.sleep(0.1)
            reintentar = input('¿Desea reintentar descargar algunos de ellos? (s/n)')

            if reintentar.lower() == 's':
                cantidad = int(input("Indique cantidad de ID's a reintentar (LIFO) o 0 para incluir todos:"))*(-1)
                # El (-1) obedece al orden LIFO (Last In First Out) en el que se reintentan los IDs no encontrados,
                # contenidos en lista_fallidos, es decir que se considera la cantidad a reintentar contando desde el
                # más reciente hasta el más antiguo, como está definido en la variable lista_fallidos_recortada
                # a continuación:

                lista_fallidos_recortada = lista_fallidos[cantidad:]
            else:
                print('* Paso omitido *')

            # A continuación se presentan algunos datos sobre la última ejecución y la información
            # presente en las bases de datos, para que el usuario pueda utilizarla para decidir
            # qué rango de ID's intentar descargar. Nótese que el usuario debe investigar previamente
            # el sitio web al cual se realizará el web scrapping, para tener una idea del máximo ID
            # que está disponible online.

            print(f'- La última actualización tuvo lugar el día {fecha}, y el tiempo de procesamiento fue de {tiempo[0]} horas -')
            time.sleep(0.1)
            explorar = input("¿Desea buscar por rango de ID's? (s/n)")

            if explorar.lower() == 's':
                print("Ingrese rango de ID's a buscar:")
                print(f'* ID mínimo descargado: {min_existente} *')
                print(f'* ID máximo descargado: {max_existente} *')
                time.sleep(0.2)
                inicio = int(input('Inicial:'))
                print(f"ID inicial: {inicio}")
                fin = int(input('Final:'))+1
                print(f"ID final: {fin-1}")

            print('Comenzando actualización...')

            tiempo_inicial_actu = time.perf_counter()

            if reintentar.lower() == 's': # Aquí comienza el reintento de fallidos, si el usuario así lo indicó.

                with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    escritor.writerow([True]) # Se marca flag de proceso iniciado, con el que, en la próxima ejecución, se comprobará si existieron interrupciones durante el proceso.

                print("Reintentando ID's fallidos...")
                encontrados = 0

                for i in lista_fallidos_recortada:

                    url = f"https://www.palermo.com.ar/es/turf/ver-carrera/{i}"
                    print(url)

                    success = False
                    while not success:

                        try:
                            response = requests.get(url)
                            response.encoding = 'utf-8'
                            # Verifica si la solicitud fue exitosa (código 200):
                            if response.status_code == 200:
                                encontrados += 1
                                # Si existe el ID, elimina el ID de la lista de fallidos:
                                lista_fallidos.remove(i)
                                # Se parsea el contenido HTML de la carrera
                                soup = BeautifulSoup(response.text, 'html.parser')
                                # Se obtiene todo el texto visible
                                texto_visible = soup.get_text(separator=' ', strip=True)
                                # Limpieza del texto: Se eliminan espacios múltiples, saltos de línea innecesarios, etc.
                                texto_limpio = re.sub(r'\s+', ' ', texto_visible)
                                # Se guarda el texto limpio en un archivo .txt
                                with open(f'./backups/carreras/{i}.txt', 'w', encoding='utf-8') as file:
                                    file.write(texto_limpio)
                                success = True # Condición para finalizar el bucle while y pasar a la siguiente iteración del bucle for.

                        except TooManyRedirects as e:
                            success = True # Si se detecta la excepción indicada, se asume que el ID no existe en la web,
                                           # por lo cual se considera que el intento fue exitoso a nivel operativo, y el ID permanece en la lista de fallidos.
                            continue # Se pasa a la siguiente iteración del bucle while finalizándolo, ya que estará la variable success en True,
                                     # pasando así a la siguiente iteración del bucle for.

                        except (RemoteDisconnected, ConnectionError, ReadTimeout) as e:

                            # Lógica para manejar errores de conexión. Se le da la opción al usuario para reintentar, o abortar el proceso.
                            # Si el usuario decide abortar el proceso, se actualizan las variables de control y se guarda
                            # en los archivos CSV de respaldo, permitiendo así no perder la información obtenida hasta el momento
                            # y evitando inconsistencias.

                            print(f"Error de desconexión: {e}")
                            print("Por favor, verificar la conexión a internet, e ingresar 's' para reintentar o 'n' para guardar el progreso y salir del programa.")
                            opcion = input('Ingrese opción (s/n):')

                            if opcion.lower() == 's':
                                continue
                            else:
                                if encontrados > 0:
                                    print(f"Se recuperaron {encontrados} carreras de la lista de ID's fallidos.")
                                    # Se exporta el nuevo csv actualizado
                                    with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
                                        escritor = csv.writer(archivo_csv)
                                        escritor.writerow(lista_fallidos)

                                    with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                                        escritor = csv.writer(archivo_csv)
                                        escritor.writerow([None]) # Se marca flag de proceso terminado

                                    tiempo_final_actu = time.perf_counter()
                                    tiempo_actu = [round((tiempo_final_actu-tiempo_inicial_actu)/3600, 2)]
                                    fecha_actu = [date.today().strftime('%d-%m-%Y')]

                                    with open('./backups/ultima_act_fecha.csv', mode='w', newline='') as archivo_csv:
                                        escritor = csv.writer(archivo_csv)
                                        escritor.writerow(fecha_actu)

                                    with open('./backups/ultima_act_tiempo.csv', mode='w', newline='') as archivo_csv:
                                        escritor = csv.writer(archivo_csv)
                                        escritor.writerow(tiempo_actu)
                                else:
                                    print("No se recuperó ninguna carrera de la lista de ID's fallidos.")
                                return

                        time.sleep(1)

                if encontrados > 0:
                    print(f"Se recuperaron {encontrados} carreras de la lista de ID's fallidos.")
                    # Se exporta el nuevo csv actualizado
                    with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
                        escritor = csv.writer(archivo_csv)
                        escritor.writerow(lista_fallidos)
                else:
                    print("No se recuperó ninguna carrera de la lista de ID's fallidos.")

                with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    escritor.writerow([None]) # Se marca flag de proceso terminado
            else:
                encontrados = 0

            # El bloque siguiente funciona de manera muy similar al anterior, con la diferencia de que es el usuario quien
            # indica los ID inicial y final a recorrer.

            if explorar.lower() == 's':
                print(f"Explorando nuevos ID's...")
                indi = 0
                exitosos = 0
                fallidos = 0
                recuperados = 0

                with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                        escritor = csv.writer(archivo_csv)
                        escritor.writerow([True])

                for i in range(inicio, fin):

                    if i in lista_existentes:
                        url = f"https://www.palermo.com.ar/es/turf/ver-carrera/{i}"
                        print(url)
                        print(f'ID {i} ya existente. Omitiendo..')
                    else:
                        url = f"https://www.palermo.com.ar/es/turf/ver-carrera/{i}"
                        print(url)

                        success = False
                        while not success:

                            try:
                                response = requests.get(url)
                                response.encoding = 'utf-8'

                                if response.status_code == 200:
                                    indi = i
                                    if str(i) in lista_fallidos:
                                        recuperados += 1
                                        lista_fallidos.remove(str(i))
                                    else:
                                        exitosos += 1

                                    soup = BeautifulSoup(response.text, 'html.parser')
                                    texto_visible = soup.get_text(separator=' ', strip=True)
                                    texto_limpio = re.sub(r'\s+', ' ', texto_visible)

                                    with open(f'./backups/carreras/{i}.txt', 'w', encoding='utf-8') as file:
                                        file.write(texto_limpio)
                                    success = True

                            except TooManyRedirects as e:
                                fallidos += 1
                                if str(i) not in lista_fallidos:
                                    lista_fallidos.append(i)
                                success = True

                            except (RemoteDisconnected, ConnectionError, ReadTimeout) as e:

                                print(f"Error de desconexión: {e}")
                                print("Por favor, verificar la conexión a internet, e ingresar 's' para reintentar o 'n' para guardar el progreso y salir del programa.")
                                opcion = input('Ingrese opción (s/n):')

                                if opcion.lower() == 's':
                                    continue
                                else:

                                    if exitosos > 0 or recuperados > 0:

                                        fecha_actu = [date.today().strftime('%d-%m-%Y')]
                                        tiempo_final_actu = time.perf_counter()
                                        tiempo_actu = [round((tiempo_final_actu-tiempo_inicial_actu)/3600, 2)]

                                        with open('./backups/ultima_act_fecha.csv', mode='w', newline='') as archivo_csv:
                                            escritor = csv.writer(archivo_csv)
                                            escritor.writerow(fecha_actu)

                                        with open('./backups/ultima_act_tiempo.csv', mode='w', newline='') as archivo_csv:
                                            escritor = csv.writer(archivo_csv)
                                            escritor.writerow(tiempo_actu)

                                    if fallidos > 0:

                                        lista_existentes_act = []
                                        lista_fallidos_purgada = []

                                        for c in os.listdir('./backups/carreras'):
                                            c = c.replace('.txt','')
                                            lista_existentes_act.append(int(c))

                                        for n in lista_fallidos:
                                            if int(n) < max(lista_existentes_act):
                                                lista_fallidos_purgada.append(n)

                                        with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
                                                escritor = csv.writer(archivo_csv)
                                                escritor.writerow(lista_fallidos_purgada)

                                    with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                                            escritor = csv.writer(archivo_csv)
                                            escritor.writerow([None])   # Se marca flag de proceso terminado

                                    print('* Actualización interrumpida *')
                                    print('Nuevas carreras descargadas:', exitosos)
                                    print('Carreras recuperadas (ex-fallidas):', encontrados + recuperados)
                                    print('Páginas no encontradas:', fallidos)
                                    print('ID máximo encontrado:', indi)
                                    print(f'Tiempo de procesamiento: {tiempo_actu[0]} horas')
                                    return
                            time.sleep(1)

                with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    escritor.writerow([None])   # Se marca flag de proceso terminado
            else:
                exitosos = 0
                fallidos = 0
                recuperados = 0
                indi = 'N/A'

            tiempo_final_actu = time.perf_counter()
            tiempo_actu = [round((tiempo_final_actu-tiempo_inicial_actu)/3600, 2)]

            if exitosos > 0 or encontrados > 0 or recuperados > 0:
                fecha_actu = [date.today().strftime('%d-%m-%Y')]

                with open('./backups/ultima_act_fecha.csv', mode='w', newline='') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    escritor.writerow(fecha_actu)

                with open('./backups/ultima_act_tiempo.csv', mode='w', newline='') as archivo_csv:
                    escritor = csv.writer(archivo_csv)
                    escritor.writerow(tiempo_actu)

            if fallidos > 0 or recuperados > 0:

                lista_existentes_act = []
                lista_fallidos_purgada = []

                for c in os.listdir('./backups/carreras'):
                    c = c.replace('.txt','')
                    lista_existentes_act.append(int(c))

                for n in lista_fallidos:
                    if int(n) < max(lista_existentes_act):
                        lista_fallidos_purgada.append(n)

                with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
                            escritor = csv.writer(archivo_csv)
                            escritor.writerow(lista_fallidos_purgada)

            print('* Actualización finalizada *')
            print('Nuevas carreras descargadas:', exitosos)
            print('Carreras recuperadas (ex-fallidas):', encontrados + recuperados)
            print('Páginas no encontradas:', fallidos)
            print('ID máximo encontrado:', indi)
            print(f'Tiempo de procesamiento: {tiempo_actu[0]} horas')

    else:
        print('- Primera ejecución -')
        print('* Creando estructura de directorios *')
        lista_fallidos = []
        indi = 0
        exitosos = 0
        fallidos = 0
        tiempo_inicial = time.perf_counter()
        os.makedirs('./backups/carreras', exist_ok=True)
        time.sleep(1)
        print('Finalizado\n')

        print("- Ingrese el rango de ID's de las carreras a buscar/descargar -")
        print("- Se estima que cada año contiene ~5600 ID's. La primera carrera de 2022 posee ID 193261 -")
        inicio = int(input('Inicial:'))
        print(f"ID inicial: {inicio}")
        fin = int(input('Final:'))+1
        print(f"ID final: {fin-1}")
        print('Comenzando descarga...')

        with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow([True])   # Se marca flag de proceso iniciado

        for i in range(inicio, fin):
            url = f"https://www.palermo.com.ar/es/turf/ver-carrera/{i}"
            print(url)

            success = False
            while not success:

                try:
                    response = requests.get(url)
                    response.encoding = 'utf-8'

                    if response.status_code == 200:
                        indi = i
                        exitosos += 1

                        soup = BeautifulSoup(response.text, 'html.parser')
                        texto_visible = soup.get_text(separator=' ', strip=True)
                        texto_limpio = re.sub(r'\s+', ' ', texto_visible)

                        with open(f'./backups/carreras/{i}.txt', 'w', encoding='utf-8') as file:
                            file.write(texto_limpio)
                        success = True

                except TooManyRedirects as e:
                    fallidos += 1
                    lista_fallidos.append(i)
                    success = True

                except (RemoteDisconnected, ConnectionError, ReadTimeout) as e:
                    print(f"Error de desconexión: {e}")
                    print("Por favor, verificar la conexión a internet, e ingresar 's' para reintentar o 'n' para guardar el progreso y salir del programa.")
                    opcion = input('Ingrese opción (s/n):')
                    if opcion == 's':
                        continue

                    else:
                        if fallidos > 0 or exitosos > 0:
                            tiempo_final = time.perf_counter()
                            tiempo = [round((tiempo_final - tiempo_inicial)/3600, 2)]
                            fecha = [date.today().strftime('%d-%m-%Y')]

                            with open('./backups/ultima_act_fecha.csv', mode='w', newline='') as archivo_csv:
                                escritor = csv.writer(archivo_csv)
                                escritor.writerow(fecha)
                            with open('./backups/ultima_act_tiempo.csv', mode='w', newline='') as archivo_csv:
                                escritor = csv.writer(archivo_csv)
                                escritor.writerow(tiempo)

                            if fallidos > 0:
                                lista_existentes_act = []
                                lista_fallidos_purgada = []

                                for c in os.listdir('./backups/carreras'):
                                    c = c.replace('.txt','')
                                    lista_existentes_act.append(int(c))

                                # El siguiente bucle for se encarga de que en la lista de fallidos no se almacenen IDs que
                                # superen el máximo ID encontrado, ya que es probable que dichos IDs sean utilizados
                                # para identificar futuras carreras. De esta forma, se logra que la información de la lista
                                # de fallidos sea más confiable.

                                for n in lista_fallidos:
                                    if int(n) < max(lista_existentes_act):
                                        lista_fallidos_purgada.append(n)

                                with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
                                    escritor = csv.writer(archivo_csv)
                                    escritor.writerow(lista_fallidos_purgada)

                        with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
                            escritor = csv.writer(archivo_csv)
                            escritor.writerow([None])   # Se marca flag de proceso finalizado

                        print('* Descarga interrumpida *')
                        print('Carreras descargadas:', exitosos)
                        print('Páginas no encontradas:', fallidos)
                        print('ID máximo encontrado:', indi)
                        print(f'Tiempo de procesamiento: {tiempo[0]} horas')
                        return

                time.sleep(1)

        tiempo_final = time.perf_counter()
        tiempo = [round((tiempo_final - tiempo_inicial)/3600, 2)]
        fecha = [date.today().strftime('%d-%m-%Y')]

        lista_fallidos_purgada = []
        for n in lista_fallidos:
            if int(n) < indi:
                lista_fallidos_purgada.append(n)

        with open('./backups/fallidos.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow(lista_fallidos_purgada)
        with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow([None])   # Se marca flag de proceso finalizado

        with open('./backups/ultima_act_fecha.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow(fecha)
        with open('./backups/ultima_act_tiempo.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow(tiempo)

        # Al ser primera ejecución, en este paso también se crea una lista vacia de IDs procesados (pasados a DF),
        # y un DF para la base general y se exportan a csv, que luego serán utilizados por la función de actualización de bases.

        procesados = []

        with open('./backups/procesados.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow(procesados)

        df_resultados = pd.DataFrame({
            'id':[],
            'nombre':[],
            'fecha':[],
            'hora':[],
            'metros':[],
            'suelo':[],
            'condicion':[],
            'posicion':[],
            'numero':[],
            'competidor':[],
            'diferencia':[],
            'peso_jockey':[],
            'peso_caballo':[],
            'pagaria':[]
            })
        df_resultados.to_csv('./backups/df_resultados.csv', index=False)

        print('* Descarga finalizada *')
        print('Carreras descargadas:', exitosos)
        print('Páginas no encontradas:', fallidos)
        print('ID máximo encontrado:', indi)
        print(f'Tiempo de procesamiento: {tiempo[0]} horas')

In [5]:
def d_c():
  """Función integradora para comprobar primero interrupciones de ejecución anteriores y corregir si hace falta,
  y posteriormente para iniciar el proceso de descarga de nuevas carreras y/o reintento de fallidos"""

  directorios, lista_fallidos, fecha, tiempo, min_existente, max_existente, lista_existentes, en_proceso = status()

  if en_proceso == True:
    print('Ejecución anterior interrumpida. Iniciando comprobación de errores...')
    time.sleep(0.2)
    lista_fallidos = comprobar_errores(lista_existentes, min_existente, max_existente)
    with open('./backups/en_proceso.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow([None])


  descargar_carreras(directorios, lista_fallidos, fecha, tiempo, min_existente, max_existente, lista_existentes)

  print('\nPresione [Enter] para volver al menú principal')
  time.sleep(0.2)
  input('')
  print('')

Funciones para actualizar la base de resultados y almacenarla en CSV:

In [6]:
# Extraer datos generales de la carrera

def ext_datos_generales(txtcarrera):
  """Esta función toma como parámetro un archivo txt con datos de una carrera, generado en la función descargar_carrera(),
  y, por medio de referencias en el texto del archivo, identifica los datos generales de la carrera y almacena en una
  variable los datos que luego se utilizarán. Los datos generales corresponden a la fecha, la hora, la distancia de la pista y su estado,
  entre otros.
  """

  contador1 = 0
  contador2 = 0

  for i in range(len(txtcarrera)):
    if txtcarrera[i] =='-':
      contador1 = i+2
      break

  carrera_dg = txtcarrera[contador1:]

  test = 0
  for i in range(len(carrera_dg)):

    if carrera_dg[i].isdigit() == True or carrera_dg[i] == ':':
      test +=1
    else:
      test = 0

    if test == 6:
      contador2 = i-6
      break

  carrera_dg = carrera_dg[:contador2]
  carrera_dg = carrera_dg.replace(' VIDEO FECHA HORA DISTANCIA PISTA TIEMPO CONDICIóN PREMIOS','')
  carrera_dg = carrera_dg.replace('| ','')

  return carrera_dg


In [7]:
def preparar_datos_gen(dg_carrera):
  """La función toma el archivo generado anteriormente, e identifica y separa la información útil en las correspondientes
  variables.
  """
  contador = 0
  test = 0

  for i in range(len(dg_carrera)):

    if dg_carrera[i].isdigit() == True or dg_carrera[i] == '/':
      test +=1
    else:
      test = 0

    if test == 10:
      contador = i-10
      break

  nombre = dg_carrera[:contador]
  fecha, hora, metros, suelo, condicion = dg_carrera[contador:].strip().split()

  return nombre, fecha, hora, metros, suelo, condicion

In [8]:
def ext_resultados(carrera):
  """Delimita el espacio en donde se encuentra la información de los resultados de la carrera, en el archivo txt pasado por
  parámetro.
  """

  x = re.search(r'Resultado ', carrera)
  inicio = x.span()[1]+95 # sumando 95 se elimina la primera repeticion de los nombres de las columnas.

  y = re.search(r' Datos del Ganador', carrera)
  fin = y.span()[0]

  extracto_1 = carrera[inicio:fin]
  extracto_1 = extracto_1.replace(' POS.. NRO. COMPETIDOR TM DISTANCIA. JOCKEY CUIDADOR CABALLERIZA. PESO JOCKEY / CABALLO PAGARIA ',';')
  extracto_1 = extracto_1.replace('FB ','')
  extracto_1 = extracto_1.replace('FB-S ','')
  extracto_1 = extracto_1.split(';')

  return extracto_1

In [9]:
def preparar_resultado(fila):
  """Se recorre cada elemento de la lista obtenida en la función ext_resultados y se seccionan y almacenan
  los datos en las variables necesarias, para luego ser agregados al DF de resultados (y por ende al archivo CSV de respaldo)
  en la función integradora a_b().
  """

  diferencias = ['PRIMER PUESTO','VENTAJA MINIMA','1 HOCICO','1/2 CABEZA','1 CABEZA','1/2 PESCUEZO','1 PESCUEZO','1/2 CUERPO','3/4 CUERPO','1 CUERPO','1 1/2 CUERPO','2 CUERPO','2 1/2 CUERPO','3 CUERPO','3 1/2 CUERPO','4 CUERPO','4 1/2 CUERPO','5 CUERPO','5 1/2 CUERPO','6 CUERPO','6 1/2 CUERPO','7 CUERPO','7 1/2 CUERPO','8 CUERPO','8 1/2 CUERPO','9 CUERPO','9 1/2 CUERPO','10 CUERPO','10 1/2 CUERPO','11 CUERPO','11 1/2 CUERPO','12 CUERPO','12 1/2 CUERPO','13 CUERPO','13 1/2 CUERPO','14 CUERPO','14 1/2 CUERPO','15 CUERPO','15 1/2 CUERPO','16 CUERPO','16 1/2 CUERPO','17 CUERPO','17 1/2 CUERPO','18 CUERPO','18 1/2 CUERPO','19 CUERPO','19 1/2 CUERPO','20 CUERPO','20 1/2 CUERPO','21 CUERPO','21 1/2 CUERPO','22 CUERPO','22 1/2 CUERPO','23 CUERPO','23 1/2 CUERPO','24 CUERPO','24 1/2 CUERPO','25 CUERPO','25 1/2 CUERPO','26 CUERPO','26 1/2 CUERPO','27 CUERPO','27 1/2 CUERPO','28 CUERPO','28 1/2 CUERPO','29 CUERPO','29 1/2 CUERPO','30 CUERPO','SIN APRECIACION','RODO','DESMONTO','DIST.MOLESTAR','DIST.DOPING']

  cont1 = 0
  for c in range(len(fila)):
    if fila[c] == ' ':
      cont1 += 1
    if cont1 == 2:
      corte1 = c
      break

  corte_pos1 = fila[0:corte1]
  posicion, numero = corte_pos1.split()
  fila = fila[corte1+1:]

  if posicion == 'RET':
    competidor = fila
    diferencia = np.nan
    peso_jockey = np.nan
    peso_caballo = np.nan
    pagaria = np.nan

  else:
      # buscar donde alguna de las palabras/frases de la lista "diferencias" y al encontrarla identificar el indice en el cual inicia dicha frase o palabra.
      patron = '|'.join(re.escape(palabra) for palabra in diferencias) # crea un patrón para buscar con el modulo regex
      match = re.search(patron, fila)  # Buscar la  coincidencia de cualquier palabra en la lista diferencias

      if match:         # Si encuentra una coincidencia, devuelve el índice de inicio y fin
        corte2 = match.span()[0]
        diferencia = match.group()
        competidor = fila[:corte2-1]
      else:
        diferencia = 'revisar error'
        competidor = 'revisar error'

      cont3 = 0
      for i in range(len(fila)-1, 0, -1):
        if fila[i] == ' ':
            cont3 += 1
        if cont3 == 2:
            corte3 = i
            break

      corte_pos2 = fila[corte3+1:]
      pesos, pagaria = corte_pos2.split()

      try:
        peso_jockey, peso_caballo = pesos.split('/')
      except ValueError:
        pesos = pagaria
        pagaria = np.nan
        peso_jockey, peso_caballo = pesos.split('/')

  return posicion, numero, competidor, diferencia, peso_jockey, peso_caballo, pagaria

In [10]:
def a_b():
  """Función integradora para actualizar la base de datos con la nueva información descargada.
  """
  # 1. se importan los csv con las listas de ID procesados y el df principal.
  with open('./backups/procesados.csv', mode='r', newline='') as archivo_csv:
        lector = csv.reader(archivo_csv)
        procesados = next(lector)

  df_resultados = pd.read_csv('./backups/df_resultados.csv', parse_dates=['fecha'])

  # 2. Compara el largo de la lista ID procesados y la cantidad de archivos .txt de carreras. Si hay más archivos que ID procesados,
  # se procede a realizar la actualización de la base de datos con aquellos archivos nuevos. En caso de no haber diferencia, se informa
  # al usuario y sale de la función.

  carreras = os.listdir('./backups/carreras')

  if len(carreras) > len(procesados):
      print('Actualizando base de datos...')
      time.sleep(1)

      # recorre el directorio de carreras, y procesa aquellos archivos que no figuran en la lista 'procesados'.

      for id in carreras:
          if id in procesados:
              continue # continua al siguiente archivo sin hacer nada más.

          else:
              print(id)
              procesados.append(id)
              idc = id.replace('.txt','')

              with open(f'./backups/carreras/{id}', 'r', encoding='utf-8') as arch:
                carrera = arch.read()

              datos_generales = ext_datos_generales(carrera)
              nombre, fecha, hora, metros, suelo, condicion = preparar_datos_gen(datos_generales)

              resultado_raw = ext_resultados(carrera)

              for fila in resultado_raw:
                posicion, numero, competidor, diferencia, peso_jockey, peso_caballo, pagaria = preparar_resultado(fila)

                # Se formatea una nueva línea para agregar al DF de resultados
                nueva_fila = pd.DataFrame({
                                        'id':[idc],
                                        'nombre':[nombre],
                                        'fecha':[fecha],
                                        'hora':[hora],
                                        'metros':[metros],
                                        'suelo':[suelo],
                                        'condicion':[condicion],
                                        'posicion':[posicion],
                                        'numero':[numero],
                                        'competidor':[competidor],
                                        'diferencia':[diferencia],
                                        'peso_jockey':[peso_jockey],
                                        'peso_caballo':[peso_caballo],
                                        'pagaria':[pagaria]
                                        })

                nueva_fila['fecha'] = pd.to_datetime(nueva_fila['fecha'], format='%d/%m/%Y')
                nueva_fila['hora'] = pd.to_datetime(nueva_fila['hora'], format='%H:%M').dt.time

                df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

      #una vez concluida la actualización, se exporta df y lista de procesados a csv.
      with open('./backups/procesados.csv', mode='w', newline='') as archivo_csv:
            escritor = csv.writer(archivo_csv)
            escritor.writerow(procesados)

      df_resultados.to_csv('./backups/df_resultados.csv', index=False)

      print('Actualización de base de datos finalizada')

  else:
      print('La base de datos está actualizada. Omitiendo paso.')
      time.sleep(1)

  print('\nPresione [Enter] para volver al menú principal')
  time.sleep(1)
  input('')
  print('')

  return df_resultados

Funciones para actualizar rankings de jockeys y cuidadores:

In [11]:
def descargar_rankings():
  """Se conecta a la URL indicada y mediante parseo del html, recupera la información actualizada de los rankings de jockeys y cuidadores.
  """

  url = f"https://www.palermo.com.ar/es/turf/ranking-de-jockeys-y-cuidadores"
  print('Descargando última versión del ranking de jockeys y cuidadores...')
  print(url)
  response = requests.get(url)
  response.encoding = 'utf-8'
  soup = BeautifulSoup(response.text, 'html.parser')
  texto_visible = soup.get_text(separator=' ', strip=True)

  with open(f'./backups/rankings.txt', 'w', encoding='utf-8') as file:
    file.write(texto_visible)

In [12]:
def generar_rankings():
  """Se crean DFs vacíos, para luego ser poblados por la información actualizada
  """

  df_jockeys = pd.DataFrame({
              'posicion':[],
              'nombre':[],
              'corr':[],
              '1ro':[],
              '2do':[],
              '3ro':[],
              '4to':[],
              '5to':[],
              'EF%_1':[],
              'EF%_2al5':[]
          })

  df_cuidadores = pd.DataFrame({
              'posicion':[],
              'nombre':[],
              'corr':[],
              '1ro':[],
              '2do':[],
              '3ro':[],
              '4to':[],
              '5to':[],
              'EF%_1':[],
              'EF%_2al5':[]
          })

  with open(f'./backups/rankings.txt', 'r', encoding='utf-8') as arch:
    rankings = arch.read()

  rankings_1 = rankings[208:-1461]

  x = re.search(r' POS CUIDADOR CORR 1ro 2do 3ro 4to 5to EF% 1 EF% 2 al 5 ', rankings_1)
  corte_i = x.span()[0]
  corte_f = x.span()[1]

  ranking_jockeys = rankings_1[:corte_i]
  ranking_cuidadores = rankings_1[corte_f:]

  return ranking_cuidadores, ranking_jockeys, df_cuidadores, df_jockeys

In [13]:
def actualizar_ranking(ranking_original, df):
  """Utilizando los rankings retornados en la función anterior, se completan los dfs vacíos.

  Argumentos:

  ranking_original -- ranking_jockeys o ranking_cuidadores
  df -- df_jockeys o df_cuidadores
  """

  for p in range(1,16):

    posicion = p

    patron = rf'{p} [A-Z]'
    rdo_busqueda = re.search(patron, ranking_original)
    corte_1 = rdo_busqueda.span()[0]

    if p < 10:
      ranking_recorte = ranking_original[corte_1+2:]
    else:
      ranking_recorte = ranking_original[corte_1+3:]

    chars = string.ascii_uppercase + ' ' + '(' + ')'
    corte_2 = 0
    for c in range(len(ranking_recorte)):
          if ranking_recorte[c] in chars:
            continue
          else:
            nombre = ranking_recorte[:c-1]
            corte_2 = c
            break
    ranking_recorte = ranking_recorte[corte_2:]

    if p < 15:
      patron = rf'{p+1} [A-Z]'
      rdo_busqueda = re.search(patron, ranking_recorte)
      corte_3 = rdo_busqueda.span()[0]-1
      ranking_recorte = ranking_recorte[:corte_3]

    valor1, valor2, valor3, valor4, valor5, valor6, valor7, valor8 = ranking_recorte.split()

    valor7 = valor7.replace(',','.')
    valor8 = valor8.replace(',','.')

    nueva_fila = pd.DataFrame({
                               'posicion':[posicion],
                                 'nombre':[nombre],
                                   'corr':[valor1],
                                    '1ro':[valor2],
                                    '2do':[valor3],
                                    '3ro':[valor4],
                                    '4to':[valor5],
                                    '5to':[valor6],
                                  'EF%_1':[valor7],
                               'EF%_2al5':[valor8]
                              })

    df = pd.concat([df, nueva_fila], ignore_index=True)

  df['posicion'] = df['posicion'].astype(int)

  return df

In [14]:
def a_r():
  """Función integradora para actualizar los archivos de respaldo de rankings.
  """

  if os.path.isfile('./backups/df_jockeys.csv'):

    with open('./backups/fecha_rankings.csv', mode='r', newline='') as archivo_csv:
      lector = csv.reader(archivo_csv)
      fecha_rankings = next(lector)

    opcion = input(f'Los rankings se actualizaron por última vez el {fecha_rankings}. ¿Desea actualizarlos ahora? (s/n)')
    time.sleep(1)

    if opcion.lower() == 'n':
       print('Omitiendo...')
       return

  print('Actualizando rankings...')
  time.sleep(1)
  descargar_rankings()

  ranking_cuidadores, ranking_jockeys, df_cuidadores, df_jockeys = generar_rankings()
  df_jockeys = actualizar_ranking(ranking_jockeys, df_jockeys)
  df_cuidadores = actualizar_ranking(ranking_cuidadores, df_cuidadores)

  df_jockeys.to_csv('./backups/df_jockeys.csv')
  df_cuidadores.to_csv('./backups/df_cuidadores.csv')
  fecha_rankings = [date.today().strftime('%d-%m-%Y')]

  with open('./backups/fecha_rankings.csv', mode='w', newline='') as archivo_csv:
    escritor = csv.writer(archivo_csv)
    escritor.writerow(fecha_rankings)

  print('Actualización de rankings finalizada.')
  time.sleep(1)
  print('\nPresione [Enter] para volver al menú principal')
  input('')
  print('')

Funciones para leer un programa de carreras dado, ubicado en "pdf_path":

In [15]:
def escanear_programa(pdf_path):
  """Se crea la Clase Dfreuniones para luego instanciar todos los dataframe donde se almacenarán los datos de
  cada próxima carrera del programa analizado, asi como una lista asociada a cada uno conteniendo el número de carrera y
  la distancia en metros. También se crea una variable conteniendo la fecha del programa analizado y el número de reunión,
  para utilizarlo posteriormente en la presentanción del resultado.

  Para recorrer el PDF se utiliza la búsqueda por coordenadas, es decir qué, al encontrarse cierta frase o texto, se
  traza un rectángulo delimitador y se utilizan dichas coordenadas para buscar otros textos utilizando esta referencia para
  ubicarlos.
  """

  class DFreuniones:
    def __init__(self):
         self.dataframe = pd.DataFrame(columns=['chaquetilla', 'competidor', 'jockey', 'edad_caballo'])

    def obtener_dataframe(self):
        return self.dataframe

  dfp_base = DFreuniones()

  dfp_1 = dfp_base.obtener_dataframe()
  # En las listas datos_carrera_#, se almacena el número de carrera y posteriormente se agregará la distancia en metros.
  # Los datos se agregan en ese orden, y se accederá luego a ellos a través de sus índices.
  datos_carrera_1 = ['1ª Carrera']

  dfp_2 = dfp_base.obtener_dataframe()
  datos_carrera_2 = ['2ª Carrera']

  dfp_3 = dfp_base.obtener_dataframe()
  datos_carrera_3 = ['3ª Carrera']

  dfp_4 = dfp_base.obtener_dataframe()
  datos_carrera_4 = ['4ª Carrera']

  dfp_5 = dfp_base.obtener_dataframe()
  datos_carrera_5 = ['5ª Carrera']

  dfp_6 = dfp_base.obtener_dataframe()
  datos_carrera_6 = ['6ª Carrera']

  dfp_7 = dfp_base.obtener_dataframe()
  datos_carrera_7 = ['7ª Carrera']

  dfp_8 = dfp_base.obtener_dataframe()
  datos_carrera_8 = ['8ª Carrera']

  dfp_9 = dfp_base.obtener_dataframe()
  datos_carrera_9 = ['9ª Carrera']

  dfp_10 = dfp_base.obtener_dataframe()
  datos_carrera_10 = ['10ª Carrera']

  dfp_11 = dfp_base.obtener_dataframe()
  datos_carrera_11 = ['11ª Carrera']

  dfp_12 = dfp_base.obtener_dataframe()
  datos_carrera_12 = ['12ª Carrera']

  dfp_13 = dfp_base.obtener_dataframe()
  datos_carrera_13 = ['13ª Carrera']

  dfp_14 = dfp_base.obtener_dataframe()
  datos_carrera_14 = ['14ª Carrera']

  dfp_15 = dfp_base.obtener_dataframe()
  datos_carrera_15 = ['15ª Carrera']

  dfp_16 = dfp_base.obtener_dataframe()
  datos_carrera_16 = ['16ª Carrera']

  doc = fitz.open(pdf_path)

  cantidad_carreras = 0
  meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

  # Iterar sobre cada página
  for pagina_num in range(doc.page_count):
    pagina = doc.load_page(pagina_num)  # Cargar la página
    texto = pagina.get_text("text")  # Obtener todo el texto de la página

    # Fórmula para pasar de milímetros a puntos PDF: (mm / 25.4) * 72

    if pagina_num == 0:
      for mes in meses:
        if re.search(mes, texto):
          coordenadas_mes = pagina.search_for(mes)
          coordenadas_mes = coordenadas_mes[0]

          # fecha que se utilizará para todas las carreras del día, al emitir el reporte.
          reunion_fecha = pagina.get_text("text", clip=fitz.Rect(coordenadas_mes.x0 - 212.60, coordenadas_mes.y0, coordenadas_mes.x1 + 73.7, coordenadas_mes.y1)).rstrip()


    for c in range(1,17):
      carrera = ''
      if c == 1:
        carrera = fr'\b{c}ª Carrera\b'
      else:
        carrera = fr'\b{c} ª Carrera\b'

      if re.search(carrera, texto):
        cantidad_carreras += 1
        carrera = carrera.replace('\\b', '')

        #detectar coordenadas de la numeración de cada carrera:
        coord_nro_carrera = pagina.search_for(carrera)
        coord_nro_carrera = coord_nro_carrera[0]

        #detectar coordenadas de la palabra "metros" para luego extraer la longitud de la pista de la carrera analizada.
        coord_metros = pagina.search_for("metros", clip=fitz.Rect(coord_nro_carrera.x0 + 76.53, coord_nro_carrera.y0 - 14.17, coord_nro_carrera.x1 + 413.86, coord_nro_carrera.y1 + 10.49))
        coord_metros = coord_metros[0]

        metros = pagina.get_text("text", clip=fitz.Rect(coord_metros.x0 - 31.18, coord_metros.y0, coord_metros.x1 - 48.19, coord_metros.y1)).rstrip()

        # Búsqueda de las coordenadas de la palabra "Caballeriza", debajo de la numeración de la carrera,
        # para ubicar donde comienza el cuadro con los datos de los competidores inscriptos.

        coord_caballeriza = pagina.search_for("Caballeriza", clip=fitz.Rect(coord_nro_carrera.x0, coord_nro_carrera.y0, coord_nro_carrera.x1, coord_nro_carrera.y1 + 159))
        coord_caballeriza = coord_caballeriza[0]

        # Una vez encontradas las coordenadas de "Caballeriza", se utilizan estas como referencia para encontrar las coordenadas del encabezado del campo "Nº"

        coord_Nro = pagina.search_for("Nº", clip=fitz.Rect(coord_caballeriza.x0, coord_caballeriza.y0, coord_caballeriza.x1 + 97, coord_caballeriza.y1))
        coord_Nro = coord_Nro[0]

        # Luego de detectar las coordinadas de "Nº", se buscan los números de chaquetilla de los primeros 2 participantes (1 y 2),
        # para calcular la distancia a la cual se encuentra cada registro, ya que puede variar de carrera a carrera (fila mas ancha o más fina)
        # Para buscar estos numeros, se prepara un rectangulo hacia abajo de "Nº" tomando como referncia los casos en donde las columnas son las más anchas.

        coord_1 = pagina.search_for("1", clip=fitz.Rect(coord_Nro.x0 - 4.25, coord_Nro.y0, coord_Nro.x1 + 4.25, coord_Nro.y1 + 56.7))
        coord_1 = coord_1[0]

        coord_2 = pagina.search_for("2", clip=fitz.Rect(coord_Nro.x0 - 4.25, coord_Nro.y0, coord_Nro.x1 + 4.25, coord_Nro.y1 + 56.7))
        coord_2 = coord_2[0]

        # A continuación se calcula la diferencia entre ambas distancias (y1 e y0) de cada nro detectado, para calcular
        # el valor de "step" que se va utilizar para ir recorriendo las filas hacia abajo y tomando los datos
        # de cada competidor. Tambien se almacenan los valores que tomará el ancho de cada recuadro.

        step_y = coord_2.y0 - coord_1.y0
        x0 = coord_Nro.x0 - 4.25
        x1 = coord_Nro.x1 + 4.25
        coords = fitz.Rect(x0, coord_1.y0, x1, coord_1.y1)  # coordenadas iniciales por donde se comenzará la búsqueda de competidores en la tabla
        continuar_bucle = True # esta variable indicará al bucle while cuando detenerse debido a que ya se procesó el último competidor de la carrera analizada

        print(carrera)

        while continuar_bucle == True:
          chaquetilla = pagina.get_text("text", clip=coords).rstrip()
          competidor_raw = pagina.get_text("text", clip=fitz.Rect(coords.x1, coords.y0, coords.x1+110, coords.y1)).rstrip()

          # se limpia el campo de competidor para dejar solo el nombre y descartar otros datos

          competidor_list = competidor_raw.strip().split()
          competidor = ''

          for i in range(len(competidor_list)):

            try:
              float(competidor_list[i])
            except:
              continue
            else:
              competidor_list.remove(competidor_list[i])

          for e in competidor_list:
            if competidor == '':
              competidor = e
            else:
              competidor = competidor + ' ' + e

          jockey = pagina.get_text("text", clip=fitz.Rect(coords.x1+110, coords.y0, coords.x1+110+89.69, coords.y1)).rstrip().upper()
          edad = pagina.get_text("text", clip=fitz.Rect(coords.x1+110+101.59, coords.y0, coords.x1+110+89.69+20.3, coords.y1)).rstrip().upper()

          # se comprueba que no se haya leido accidentalemente el color del pelaje junto con la edad, y se procede a limpiar si ese es el caso:

          if len(edad) == 3:
            edad = edad[2]
          elif len(edad) == 2:
            edad = edad[1]

          nueva_fila = pd.DataFrame({
                                        'chaquetilla':[chaquetilla],
                                        'competidor':[competidor],
                                        'jockey':[jockey],
                                        'edad_caballo':[edad],
                                        })

          match c:
            case (1):
              dfp_1 = pd.concat([dfp_1, nueva_fila], ignore_index=True)
              if len(datos_carrera_1) == 1:
                datos_carrera_1.append(metros)
            case (2):
              dfp_2 = pd.concat([dfp_2, nueva_fila], ignore_index=True)
              if len(datos_carrera_2) == 1:
                datos_carrera_2.append(metros)
            case (3):
              dfp_3 = pd.concat([dfp_3, nueva_fila], ignore_index=True)
              if len(datos_carrera_3) == 1:
                datos_carrera_3.append(metros)
            case (4):
              dfp_4 = pd.concat([dfp_4, nueva_fila], ignore_index=True)
              if len(datos_carrera_4) == 1:
                datos_carrera_4.append(metros)
            case (5):
              dfp_5 = pd.concat([dfp_5, nueva_fila], ignore_index=True)
              if len(datos_carrera_5) == 1:
                datos_carrera_5.append(metros)
            case (6):
              dfp_6 = pd.concat([dfp_6, nueva_fila], ignore_index=True)
              if len(datos_carrera_6) == 1:
                datos_carrera_6.append(metros)
            case (7):
              dfp_7 = pd.concat([dfp_7, nueva_fila], ignore_index=True)
              if len(datos_carrera_7) == 1:
                datos_carrera_7.append(metros)
            case (8):
              dfp_8 = pd.concat([dfp_8, nueva_fila], ignore_index=True)
              if len(datos_carrera_8) == 1:
                datos_carrera_8.append(metros)
            case (9):
              dfp_9 = pd.concat([dfp_9, nueva_fila], ignore_index=True)
              if len(datos_carrera_9) == 1:
                datos_carrera_9.append(metros)
            case (10):
              dfp_10 = pd.concat([dfp_10, nueva_fila], ignore_index=True)
              if len(datos_carrera_10) == 1:
                datos_carrera_10.append(metros)
            case (11):
              dfp_11 = pd.concat([dfp_11, nueva_fila], ignore_index=True)
              if len(datos_carrera_11) == 1:
                datos_carrera_11.append(metros)
            case (12):
              dfp_12 = pd.concat([dfp_12, nueva_fila], ignore_index=True)
              if len(datos_carrera_12) == 1:
                datos_carrera_12.append(metros)
            case (13):
              dfp_13 = pd.concat([dfp_13, nueva_fila], ignore_index=True)
              if len(datos_carrera_13) == 1:
                datos_carrera_13.append(metros)
            case (14):
              dfp_14 = pd.concat([dfp_14, nueva_fila], ignore_index=True)
              if len(datos_carrera_14) == 1:
                datos_carrera_14.append(metros)
            case (15):
              dfp_15 = pd.concat([dfp_15, nueva_fila], ignore_index=True)
              if len(datos_carrera_15) == 1:
                datos_carrera_15.append(metros)
            case (16):
              dfp_16 = pd.concat([dfp_16, nueva_fila], ignore_index=True)
              if len(datos_carrera_16) == 1:
                datos_carrera_16.append(metros)

          # ahora se comprueba si la siguiente linea contiene otro competidor. En dicho caso actualiza el rectangulo "coords"
          # y continua el bucle. Caso contrario, se cambia el valor de la var. continuar bucle para detener el proceso y pasar a la siguiente carrera

          validacion = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']
          comprobacion = pagina.get_text("text", clip=fitz.Rect(coords.x0, coords.y0+step_y, coords.x1, coords.y1+step_y)).rstrip().strip()

          if comprobacion in validacion:
              coords = fitz.Rect(coords.x0, coords.y0+step_y, coords.x1, coords.y1+step_y)
          else:
              continuar_bucle = False

  return reunion_fecha, cantidad_carreras, dfp_1, datos_carrera_1, dfp_2, datos_carrera_2, dfp_3, datos_carrera_3, dfp_4, datos_carrera_4, dfp_5, datos_carrera_5, dfp_6, datos_carrera_6, dfp_7, datos_carrera_7, dfp_8, datos_carrera_8, dfp_9, datos_carrera_9, dfp_10, datos_carrera_10, dfp_11, datos_carrera_11, dfp_12, datos_carrera_12, dfp_13, datos_carrera_13, dfp_14, datos_carrera_14, dfp_15, datos_carrera_15, dfp_16, datos_carrera_16

In [16]:
def l_p():
  """Función integradora para escanear programas de futuras carreras
  """

  print('Programas/rutas disponibles:')

  for f in os.listdir('./calendario'):
    print(f'./calendario/{f}')

  programa = input('Copie y pegue la ruta del programa a escanear del listado anterior:')

  reunion_fecha, cantidad_carreras, dfp_1, datos_carrera_1, dfp_2, datos_carrera_2, dfp_3, datos_carrera_3, dfp_4, datos_carrera_4, dfp_5, datos_carrera_5, dfp_6, datos_carrera_6, dfp_7, datos_carrera_7, dfp_8, datos_carrera_8, dfp_9, datos_carrera_9, dfp_10, datos_carrera_10, dfp_11, datos_carrera_11, dfp_12, datos_carrera_12, dfp_13, datos_carrera_13, dfp_14, datos_carrera_14, dfp_15, datos_carrera_15, dfp_16, datos_carrera_16 = escanear_programa(programa)

  print(f'{reunion_fecha}')
  print(f'Se correrán {cantidad_carreras} carreras en esta reunión.')
  print('\nElija a continuación el nro. de carrera deseado para ver la información:')

  while True:
    carrera = input(f'Ingrese un valor del 1 al {cantidad_carreras} (total de carreras del programa), o presione enter para volver al menú principal')
    time.sleep(0.2)

    if carrera == '':
      break

    if carrera.isdigit():
        carrera = int(carrera)
        if carrera > cantidad_carreras:
          print('El número ingresado excede la cantidad de carreras del programa.')
        else:
          if carrera == 1:
            print(f'\nLa {datos_carrera_1[0]} será de {datos_carrera_1[1]} metros.')
            print('\nParticipantes:')
            display(dfp_1)
            input('Presione [Enter] para continuar')

          elif carrera == 2:
            print(f'\nLa {datos_carrera_2[0]} será de {datos_carrera_2[1]} metros.')
            print('\nParticipantes:')
            display(dfp_2)
            input('Presione [Enter] para continuar')

          elif carrera == 3:
            print(f'\nLa {datos_carrera_3[0]} será de {datos_carrera_3[1]} metros.')
            print('\nParticipantes:')
            display(dfp_3)
            input('Presione [Enter] para continuar')

          elif carrera == 4:
            print(f'\nLa {datos_carrera_4[0]} será de {datos_carrera_4[1]} metros.')
            print('\nParticipantes:')
            display(dfp_4)
            input('Presione [Enter] para continuar')

          elif carrera == 5:
            print(f'\nLa {datos_carrera_5[0]} será de {datos_carrera_5[1]} metros.')
            print('\nParticipantes:')
            display(dfp_5)
            input('Presione [Enter] para continuar')

          elif carrera == 6:
            print(f'\nLa {datos_carrera_6[0]} será de {datos_carrera_6[1]} metros.')
            print('\nParticipantes:')
            display(dfp_6)
            input('Presione [Enter] para continuar')

          elif carrera == 7:
            print(f'\nLa {datos_carrera_7[0]} será de {datos_carrera_7[1]} metros.')
            print('\nParticipantes:')
            display(dfp_7)
            input('Presione [Enter] para continuar')

          elif carrera == 8:
            print(f'\nLa {datos_carrera_8[0]} será de {datos_carrera_8[1]} metros.')
            print('\nParticipantes:')
            display(dfp_8)
            input('Presione [Enter] para continuar')

          elif carrera == 9:
            print(f'\nLa {datos_carrera_9[0]} será de {datos_carrera_9[1]} metros.')
            print('\nParticipantes:')
            display(dfp_9)
            input('Presione [Enter] para continuar')

          elif carrera == 10:
            print(f'\nLa {datos_carrera_10[0]} será de {datos_carrera_10[1]} metros.')
            print('\nParticipantes:')
            display(dfp_10)
            input('Presione [Enter] para continuar')

          elif carrera == 11:
            print(f'\nLa {datos_carrera_11[0]} será de {datos_carrera_11[1]} metros.')
            print('\nParticipantes:')
            display(dfp_11)
            input('Presione [Enter] para continuar')

          elif carrera == 12:
            print(f'\nLa {datos_carrera_12[0]} será de {datos_carrera_12[1]} metros.')
            print('\nParticipantes:')
            display(dfp_12)
            input('Presione [Enter] para continuar')

          elif carrera == 13:
            print(f'\nLa {datos_carrera_13[0]} será de {datos_carrera_13[1]} metros.')
            print('\nParticipantes:')
            display(dfp_13)
            input('Presione [Enter] para continuar')

          elif carrera == 14:
            print(f'\nLa {datos_carrera_14[0]} será de {datos_carrera_14[1]} metros.')
            print('\nParticipantes:')
            display(dfp_14)
            input('Presione [Enter] para continuar')

          elif carrera == 15:
            print(f'\nLa {datos_carrera_15[0]} será de {datos_carrera_15[1]} metros.')
            print('\nParticipantes:')
            display(dfp_15)
            input('Presione [Enter] para continuar')

          elif carrera == 16:
            print(f'\nLa {datos_carrera_16[0]} será de {datos_carrera_16[1]} metros.')
            print('\nParticipantes:')
            display(dfp_16)
            input('Presione [Enter] para continuar')
    else:
      print('El valor ingresado no es un número.')


Función para exportar las bases de datos a archivos de Excel:

In [17]:
def e_i():

    fecha = datetime.now().date()

    df_resultados = pd.read_csv("./backups/df_resultados.csv")
    df_jockeys = pd.read_csv("./backups/df_jockeys.csv")
    df_cuidadores = pd.read_csv("./backups/df_cuidadores.csv")

    print('\nExportando base de resultados...')
    df_resultados.to_excel(f"./exports/base_resultados {fecha}.xlsx", index=False)
    print('Exportando ranking de jockeys...')
    df_jockeys.to_excel(f"./exports/ranking_jockeys {fecha}.xlsx", index=False)
    print('Exportando ranking de cuidadores...')
    df_cuidadores.to_excel(f"./exports/ranking_cuidadores {fecha}.xlsx", index=False)
    print('Proceso finalizado')
    return

Menú principal:

In [18]:
def menu():

    while True:
        print('Turf Scrapping - Menú principal')
        print('-------------------------------')
        print('1) Descargar resultados')
        print('2) Actualizar base de resultados')
        print('3) Actualizar rankings de jockeys y cuidadores')
        print('4) Escanear programa')
        print('5) Exportar información a archivos Excel')
        print('6) Salir')
        time.sleep(0.2)
        opcion = int((input('Ingresar opción: ')))

        if opcion == 1:     # Se reconoce la opción ingresada y se ejecuta la función solicitada por el usuario.
            d_c()
        elif opcion == 2:
            a_b()
        elif opcion == 3:
            a_r()
        elif opcion == 4:
            l_p()
        elif opcion == 5:
            e_i()
        elif opcion == 6:
            print('\nSaliendo...')
            time.sleep(0.15)
            break
        else:
            print('\nError: La opción ingresada no es válida. Intente nuevamente\n')


In [19]:
menu()

Turf Scrapping - Menú principal
-------------------------------
1) Descargar resultados
2) Actualizar base de resultados
3) Actualizar rankings de jockeys y cuidadores
4) Escanear programa
5) Exportar información a archivos Excel
6) Salir
- Actualización de archivos -
Actualmente existen 12146 ID's de carreras no encontrados
* Paso omitido *
- La última actualización tuvo lugar el día 11-01-2025, y el tiempo de procesamiento fue de 0.88 horas -
Ingrese rango de ID's a buscar:
* ID mínimo descargado: 193092 *
* ID máximo descargado: 209985 *
ID inicial: 209758
ID final: 209786
Comenzando actualización...
Explorando nuevos ID's...
https://www.palermo.com.ar/es/turf/ver-carrera/209758
https://www.palermo.com.ar/es/turf/ver-carrera/209759
ID 209759 ya existente. Omitiendo..
https://www.palermo.com.ar/es/turf/ver-carrera/209760
ID 209760 ya existente. Omitiendo..
https://www.palermo.com.ar/es/turf/ver-carrera/209761
ID 209761 ya existente. Omitiendo..
https://www.palermo.com.ar/es/turf/ver-

,chaquetilla,competidor,jockey,edad_caballo
0,1,SUPER JANE,ARMOHA LEOPOLDO M,6
1,2,LA TABLET DE PLATA,GONZALEZ LUCAS F 4,7
2,3,BREEZA MANI,ORTEGA PAVON EDUARDO,6
3,4,VENUS CUYEN,CALVENTE GUSTAVO E,6
4,5,ELLAS LAS NADIES,JURI FABIAN R,6
5,6,ALMA RUNNER,BAEZ TOMAS D 3,7
6,7,KILL MODEL,SOSA MIGUEL A O,7
7,8,VIOLENTA STORM,ARREGUY FRANCISCO A (H),6
8,9,ALEGRIA JOHAN,ASERITO MAXIMILIANO,7


Turf Scrapping - Menú principal
-------------------------------
1) Descargar resultados
2) Actualizar base de resultados
3) Actualizar rankings de jockeys y cuidadores
4) Escanear programa
5) Exportar información a archivos Excel
6) Salir

Exportando base de resultados...
Exportando ranking de jockeys...
Exportando ranking de cuidadores...
Proceso finalizado
Turf Scrapping - Menú principal
-------------------------------
1) Descargar resultados
2) Actualizar base de resultados
3) Actualizar rankings de jockeys y cuidadores
4) Escanear programa
5) Exportar información a archivos Excel
6) Salir

Saliendo...
